# Описание проекта: Исследование надежности заемщиков.

Заказчик — кредитный отдел банка.


 

**Цель исследования** — проверить гипотезу:
Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

**Ход исследования**
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

 Смотрим первые 10 строк в таблице

In [1]:
import pandas as pd
#data = pd.read_csv('/datasets/data.csv')
#print(data.sort_values(by='days_employed'))

In [2]:
data = pd.read_csv('/datasets/data.csv')
print(data.sort_values(by='days_employed'))

       children  days_employed  dob_years education  education_id  \
16335         1  -18388.949901         61   среднее             1   
4299          0  -17615.563266         61   среднее             1   
7329          0  -16593.472817         60    высшее             0   
17838         0  -16264.699501         59   среднее             1   
16825         0  -16119.687737         64   среднее             1   
...         ...            ...        ...       ...           ...   
21489         2            NaN         47   Среднее             1   
21495         1            NaN         50   среднее             1   
21497         0            NaN         48    ВЫСШЕЕ             0   
21502         1            NaN         42   среднее             1   
21510         2            NaN         28   среднее             1   

          family_status  family_status_id gender income_type  debt  \
16335   женат / замужем                 0      F   сотрудник     0   
4299    женат / замужем        

Почему-то в столбце со стажем работы присутствуют отрицательные значения. 
В столбцах о стаже работы и ежемесячном доходе имеются пропуски типа NaN

Общая информация по датафрейму: стобцам, типам данных, количеству записей 

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


проверяем на наличие пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

пропуски в наличии в двух столбцах в одинаковом количестве 2174 штуки, поэтому вероятно встречаются одновременно в строке

In [5]:
#len(data['days_employed'].unique())
data[(data['days_employed'].isna() & data['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Количество "нанов" не изменилось. так же 2174 штуки. Значит пропуски в строке встерчаются одновременно и в столбце days_employed, и в столбце total_income

Доля нанов в таблице составляет:


In [6]:
(1-(data['days_employed'].count()/data['education_id'].count()))*100

10.099883855981417

10 процентов не указано о стаже работы и ежемесячном доходе, существенное значение.

In [7]:
data.isna().mean() # код по твоему совету)

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Средний возраст клиента, который не указал данных о доходе и стаже

In [8]:
int(data[data['days_employed'].isna()]['dob_years'].mean())

43

43 года - сознательные граждане, странно

In [9]:
data[data['days_employed'].isna()]['dob_years'].max() 

73

максимальный возраст заемщика 73 года

In [10]:
data[data['days_employed'].isna()]['dob_years'].min() 

0

кто - то не указал, либо указал 0 в графе возраст

In [11]:
data[(data['days_employed'].isna()) & (data['dob_years']==0)]['dob_years'].count()

10

Таких 10 человек

In [12]:
(data[(data['days_employed'].isna()) & (data['dob_years']==0)]['dob_years'].count()/data['days_employed'].count())*100

0.0516769159216578

половина сотой процента. думаю вряд ли это существенно скажется на среднем возрасте

In [13]:
len(data[data['dob_years'] == 0])


101

всего в датасете всего 101 нулевое значение в столбце возраста

In [14]:
(data[(data['dob_years']==0)]['dob_years'].count()/data['days_employed'].count())*100

0.5219368508087437

Во всем дата фрейме пол процента записей имеет значение 0 в графе возраста.


In [15]:
data.duplicated().sum()

54

Имеются явные дубликаты в количестве 54 штук

In [16]:
data.describe() # по твоему совету пробую метод describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Шаг 2.1 Заполнение пропусков

In [17]:
data.loc[data['days_employed'] != 'NaN']['days_employed'].min()



/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


-18388.949900568383

In [18]:
days_employed_median = int(data.loc[data['days_employed'] !='NaN']['days_employed'].abs().median())
days_employed_median # переменная с медианным значением стажа

2194

В столбце о стаже рабты имются отрицательные значения, видимо это дефис, с учетом этого медианное значение в стаже 2609 дней


In [19]:
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

In [20]:
data['days_employed'].isna().sum()

0

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [22]:
data['total_income'].min()

20667.26379327158

In [23]:
data['total_income'].max()

2265604.028722744

In [24]:
total_income_median = data['total_income'].median()
total_income_median

145017.93753253992

In [25]:
data['total_income'] = data['total_income'].fillna(total_income_median)
#data['total_income'].isna().sum()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [26]:
data['total_income'].isna().sum()

0

заменили пропуски на медианные значения. пропусков не осталось

### Шаг 2.2 Проверка данных на аномалии и исправления.

Аномалии: столбец со стажем (days_employed) и столбец с ежемесячным доходом (total_income) содержит отрицательные значения - видимо это дефис.


In [27]:
data.min()

children                               -1
days_employed                    -18388.9
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

Видим анмальные минимальные значения в столбце 'children' в виде отрицательного значения, и в столбце 'days_employed' тоже в виде отрицательного значения

In [28]:
data.max()

children                         20
days_employed                401755
dob_years                        75
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                          XNA
income_type                 студент
debt                              1
total_income             2.2656e+06
purpose             сыграть свадьбу
dtype: object

Видим аномальные максимальные значения в столбцах gender, days_employed, children


In [29]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [30]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'M'

In [31]:
data['gender'].unique()# проверка пола на аномалии

array(['F', 'M'], dtype=object)

In [32]:
data['children'].unique() # проверка количества детей на аномалии

array([ 1,  0,  3,  2, -1,  4, 20,  5])

значения -1 и 20 наверняка ошибочны, по каким то причинам люди их так указали, чтоже, 

In [33]:
#data.pivot_table(index=['debt','children'], columns='purpose_category', values='dob_years', aggfunc='mean')
data.pivot_table(index=['debt','children'], columns='purpose', values='dob_years', aggfunc='mean')

purpose        автомобили  автомобиль  высшее образование  \
debt children                                               
0    -1         28.000000   42.000000           33.000000   
      0         46.986159   47.281553           46.351254   
      1         38.900990   37.405941           37.795455   
      2         35.714286   34.470588           35.272727   
      3         37.714286   38.400000           36.833333   
      4               NaN   30.500000           34.000000   
      5               NaN         NaN                 NaN   
      20        69.000000   51.000000           41.000000   
1    -1               NaN   33.000000                 NaN   
      0         44.178571   43.766667           40.458333   
      1         42.777778   36.000000           39.416667   
      2         31.571429   33.750000           33.000000   
      3               NaN         NaN                 NaN   
      4               NaN         NaN                 NaN   
      20              NaN         NaN                 NaN   

purpose        дополнительное образование      жилье  \
debt children                                          
0    -1                         55.500000  37.000000   
      0                         48.099631  45.664083   
      1                         39.876543  37.961832   
      2                         35.610169  35.095238   
      3                         33.375000  36.692308   
      4                               NaN  29.000000   
      5                               NaN  35.000000   
      20                        41.000000  34.750000   
1    -1                               NaN        NaN   
      0                         43.538462  42.129032   
      1                         33.800000  31.375000   
      2                         33.714286  39.166667   
      3                               NaN        NaN   
      4                               NaN  33.000000   
      20                              NaN        NaN   

purpose        заняться высшим образованием  заняться образованием  \
debt children                                                        
0    -1                                 NaN                    NaN   
      0                           45.955782              47.876543   
      1                           40.217822              37.494253   
      2                           36.068182              36.314286   
      3                           38.181818              35.000000   
      4                           41.000000              34.500000   
      5                           38.000000                    NaN   
      20                                NaN                    NaN   
1    -1                                 NaN                    NaN   
      0                           43.714286              45.950000   
      1                           33.777778              36.769231   
      2                           33.000000              34.000000   
      3                                 NaN              38.000000   
      4                                 NaN                    NaN   
      20                                NaN                    NaN   

purpose        на покупку автомобиля  на покупку подержанного автомобиля  \
debt children                                                              
0    -1                    30.000000                           34.000000   
      0                    46.496528                           47.275510   
      1                    39.215054                           39.628866   
      2                    36.179487                           36.823529   
      3                    37.333333                           36.900000   
      4                    37.333333                           34.000000   
      5                          NaN                                 NaN   
      20                    0.000000                           31.600000   
1    -1                          NaN                      

Люди, указавшие количество детей -1 и 20 в подавляющем большинстве не имеют задолженностей по кретиту.
объединю их в группу с 0 детей - она самая большая, там незаметно будет

в датасете 47 значений "-1" в столбце children

In [34]:
data.loc[(data['children'] == 20) | (data['children'] == -1), 'children']=0

Аномальных значений существенно в столбце.

In [35]:
data.loc[(data['children'] == -1) | (data['children'] == 20)]['children'].count()

0

In [36]:
data['children'].unique() # проверка количества детей на аномалии

array([1, 0, 3, 2, 4, 5])

аномальных пограничных значений количества детей не осталось

### Шаг 2.3. Изменение типов данных.

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [38]:
data['total_income'] = data['total_income'].astype(int)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

Удаление явных дубликатов

In [39]:
data.duplicated().sum()

54

In [40]:
data = data.drop_duplicates().reset_index(drop=True)

In [41]:
data.duplicated().sum()

0

Явные дубикаты удалены. осталось найти неявные


In [42]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [43]:
data['education'] = data['education'].str.lower()


In [44]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Приведение к нижнему регистру сократило количество неявных дубликатов.

На всякий случай еще раз проверим на явные дубликаты, если таковые найдутся - удаляем.

In [45]:
data.duplicated().sum()

17

In [46]:
data = data.drop_duplicates().reset_index(drop=True)

In [47]:
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [48]:
data_education = data[['education_id','education']]
data_education

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
...,...,...
21449,1,среднее
21450,1,среднее
21451,1,среднее
21452,1,среднее


In [49]:
data_education = data_education.drop_duplicates().reset_index(drop=True)

In [50]:
data_education.duplicated().sum()

0

In [51]:
data_family_status = data[['family_status_id','family_status']]
data_family_status.duplicated().sum()

21449

In [52]:
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True)

In [53]:
data_family_status.duplicated().sum()

0

In [54]:
try:
    data = data.drop(columns=['education', 'family_status'])
except:
    print('Столбцы отсутствуют')

In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 10 columns):
children            21454 non-null int64
days_employed       21454 non-null float64
dob_years           21454 non-null int64
education_id        21454 non-null int64
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(3)
memory usage: 1.6+ MB


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

In [56]:
def income_category_simbol(row):
    total_income = row['total_income']
    try:
        if 0<total_income<=30000:
            return 'E'
        elif 30001<=total_income<=50000:
            return 'D'
        elif 50001<=total_income<=200000:
            return 'C'
        elif 200001<=total_income<=1000000:
            return 'B'
        elif total_income>= 1000001:
            return 'A'
    except:
        'Отрицательный доход'
data['total_income_category'] = data.apply(income_category_simbol, axis=1)


Функция добавления категории заемщику исходя из его дохода применилась к дата сету.
проверка результатов. посмотрим в датасете минимально знечание дохода у полученной категории.

In [57]:
data[(data['total_income'] <= 10000000) & (data['total_income']>= 0)]['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

In [58]:
data[data['total_income_category'] == 'E']['total_income'].min()

20667

In [59]:
data[data['total_income_category'] == 'D']['total_income'].min()

30075

In [60]:
data[data['total_income_category'] == 'C']['total_income'].min()

50025

In [61]:
data[data['total_income_category'] == 'B']['total_income'].min()

200001

In [62]:
data[data['total_income_category'] == 'A']['total_income'].min()

1004476

Категории расставились верно согласно начальному условию

### Шаг 2.7. Категоризация целей кредита.

посмотрим уникальные значения в столбце с целью кредита

In [63]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Много неявных дублей, можно свести к 4 целям:
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [64]:
#с помощю функции purpose_arrange, которая описана ниже, производим категоризацию целей кредита
def purpose_arrange(row):
    purpose_local = row['purpose']
    try:
        if 'автомобил' in purpose_local:
            return 'операции с автомобилем'
        elif 'недвижимост' in purpose_local:
            return 'операции с недвижимостью'
        elif ('жиль') in purpose_local:
            return 'операции с недвижимостью'
        elif 'свадьб' in purpose_local:
            return 'проведение свадьбы'
        elif 'образовани' in purpose_local:
            return 'получение образования'
    except:
        'Ошибка'

        
data['purpose_category'] = data.apply(purpose_arrange, axis=1)



In [65]:
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21451,1,-2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21452,3,-3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


### Ответы на вопросы.

##### Вопрос 1: 

Есть ли зависимость между количеством детей и возвратом кредита в срок?


Что бы ответить на этот вопрос, надо 

In [66]:
data1 = data.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count')

In [67]:
data1['total_avg'] = data1[1]/data1[0]*100


In [68]:
data1['percernt_cildren_avg'] = data1[1]/(data1[1] + data1[0])*100 #Добавленый код после первой проверки

In [69]:
data1.sort_values(by='total_avg')

debt,0,1,total_avg,percernt_cildren_avg
children,,,,
0,13142.0,1072.0,8.157054,7.541860
3,303.0,27.0,8.910891,8.181818
1,4364.0,444.0,10.174152,9.234609
2,1858.0,194.0,10.441335,9.454191
4,37.0,4.0,10.810811,9.756098
5,9.0,NaN,NaN,NaN


Самая лучшая ситуация с обслуживанием задолженности у двух групп людей: с 5 детьми и без детей

##### Вывод 1:

Из сводной таблицы видно, что заемщики с 5 детьми не допускают просрочек.
Правда таких  людей немного, поэтому не особоенно показательный результат,
но на втором месте люди без детей.
То есть можно сказать, что появление ребенка сказывается на платежеспосоности клиента

##### Вопрос 2: 

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [70]:
data3=data.pivot_table(index='family_status_id',columns='debt', values='total_income',aggfunc='count')

In [71]:
data3['total_avg']=data3[1]/data3[0]*100

In [72]:
data3['percernt_cildren_avg'] = data3[1]/(data3[1] + data3[0])*100 #Добавленый код после первой проверки

In [73]:
data3.sort_values(by='total_avg')

debt,0,1,total_avg,percernt_cildren_avg
family_status_id,,,,
2,896,63,7.031250,6.569343
3,1110,85,7.657658,7.112971
0,11408,931,8.160940,7.545182
1,3763,388,10.310922,9.347145
4,2536,274,10.804416,9.750890


In [74]:
data3.merge(data_family_status, on='family_status_id', how='left').sort_values(by='total_avg')

,family_status_id,0,1,total_avg,percernt_cildren_avg,family_status
2,2,896,63,7.031250,6.569343,вдовец / вдова
3,3,1110,85,7.657658,7.112971,в разводе
0,0,11408,931,8.160940,7.545182,женат / замужем
1,1,3763,388,10.310922,9.347145,гражданский брак
4,4,2536,274,10.804416,9.750890,Не женат / не замужем


##### Вывод 2:

Наблюдается следующая зависимость, что человек, перенесший расставание с супругом - он более ответственный в платежной дисциплине, и меньше допускает просрочек по кредиту.

##### Вопрос 3: 

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [75]:
data4 = data.pivot_table(index='total_income_category',columns = 'debt', values='total_income', aggfunc='count')

In [76]:
data4['total_avg'] = data4[1]/data4[0]*100

In [77]:
data4['percernt_cildren_avg'] = data4[1]/(data4[1] + data4[0])*100 #Добавленый код после первой проверки

In [78]:
data4.sort_values(by='total_avg')

debt,0,1,total_avg,percernt_cildren_avg
total_income_category,,,,
D,329,21,6.382979,6.000000
B,4685,356,7.598719,7.062091
A,23,2,8.695652,8.000000
C,14656,1360,9.279476,8.491508
E,20,2,10.000000,9.090909


##### Вывод 3:

Лучшая ситуация в категориях заемщиков D и B, то есть с доходами от 30001 до 100000 руб

Поправка: 

Клинеты группы D с доходом от 30 до 50 тыс в мес

Клиенты группы B с доходом от 200 до миллиона в мес

##### Вопрос 4: 

Как разные цели кредита влияют на его возврат в срок?

In [79]:
data5=data.pivot_table(index='purpose_category',columns='debt', values='dob_years', aggfunc='count')

In [80]:
data5['total_avg']=data5[1]/data5[0]*100

In [81]:
data5['percernt_cildren_avg'] = data5[1]/(data5[1] + data5[0])*100 #Добавленый код после первой проверки

In [82]:
data5.sort_values(by='total_avg')

debt,0,1,total_avg,percernt_cildren_avg
purpose_category,,,,
операции с недвижимостью,10029,782,7.797388,7.233373
проведение свадьбы,2138,186,8.699719,8.003442
получение образования,3643,370,10.156464,9.220035
операции с автомобилем,3903,403,10.325391,9.359034


##### Вывод 4:

Значимые цели кредита имеют лучшую статистику по своевременным оплатам.

## Общий вывод:

Подводя итог, можно сказать, что наиболее ответственно подходят к платежной дисциплине по кредиту люди разведенные или овдовевшие, что касается наличия детей: судя по данным видно, что лучше всего справляются с те, у кого 5 детей, а следующие за ними - люди, не имеющие ни одного ребенка. 